# Econometria de Séries Temporais

## O modelo de vetores autorregressivos

João Ricardo Costa Filho \\
[joaocostafilho.com](https://)

**Motivação**: \\

 Como projetar a inflação de preços livres? Vamos utilizar um dos modelos de vetores autorregressivos do [box do Relatório de Inflação do BCB - jun/2004](https://www.bcb.gov.br/content/ri/relatorioinflacao/200406/RELINF200406-ri200406b8p.pdf) para responder essa pergunta em duas partes. Primeiro, neste notebook, vamos estimar os modelos VAR.

# Taxas de inflação: preços livres e preços administrados

Acesse [https://www.bcb.gov.br/](https://www.bcb.gov.br/) > Estatísticas > Séries Temporais (SGS) > Atividade econômica > Preços > Índices de preços ao consumidor e encontre o código para os dados da variação mensal (i) dos preços livres e (ii) dos preços administrados. Obtenha as séries para os dois indicadores à partir de janeiro de 2004.

Faça um gráfico para cada série:

## Testes de raiz unitária

Teste se as séries são estacionárias. Para isso, utilize os testes ADF, KPSS e PP.

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



### ADF (Augmented-Dickey-Fuller)

$H_0$:

Como interpretar?

* Primeiro (antes de fazer o teste!) temos que escolher o nível de significância (1%, 5% ou 10%).
* Depois, comparamos as estatísticas calculadas (value of test-statistics) com os valores críticos:
  * tau3: hipótese nula: raiz unitária.
  * phi2: hipótese nula: raiz unitária, sem tendência e sem drift.
  * phi3: hipótese nula: raiz unitária e sem tendência.

Qual é a conclusão do teste?

R:

Qual é a conclusão do teste?

R:

### KPSS

$H_0$:

Qual é a conclusão do teste?

R:

### PP

$H_0$:

Qual é a conclusão do teste?

R:

Qual é a sua conclusão sobre as séries?

R:

# Taxa de câmbio nominal (BRL/USD)

Acesse [https://www.bcb.gov.br/](https://www.bcb.gov.br/) > Estatísticas > Séries Temporais (SGS) e utilize a busca textual (lembre-se de selecionar a periodicidade) para encontrar o código para a média mensal da taxa de câmbio de venda do dólar americano (cuja fonte é o BCB-DSTAT). Obtenha as séries para os dois indicadores desde dezembro de 2003.

Utilize a função 'diff()' para calcular a primeira diferença da série:

Faça um gráfico da série:

## Testes de raiz unitária

Teste se as séries são estacionárias. Para isso, utilize os testes ADF, KPSS e PP.

#### ADF (Augmented-Dickey-Fuller)

$H_0$:

Qual é a conclusão do teste?

R:

### KPSS

$H_0$:

Qual é a conclusão do teste?

R:

### PP

$H_0$:

Qual é a conclusão do teste?

R:

# Taxa de juros nominal - Selic

Acesse [https://www.bcb.gov.br/](https://www.bcb.gov.br/) > Estatísticas > Séries Temporais (SGS) > Mercados financeiros e de capitais > Indicadores de mercado financeiro > Taxas de juros e encontre o código para a Taxa Selic acumulada no mês (anualizada). Obtenha a série desde dezembro de 2003.

Faça um gráfico da série:

# Expectativas de inflação

Semanalmente, o BCB coleta informações sobre as chamadas [expectativas de mercado](https://www.bcb.gov.br/controleinflacao/expectativasmercado). Vamos utilizá-las no cálculo da taxa de juros real com base na equação de Fisher. Para isso, primeiro, obtenha as expectativas com base nos códigos abaixo:

In [ ]:
library(xts)

In [ ]:
ipcae <- get_twelve_months_inflation_expectations('IPCA', start_date = '2003-12-01')

In [ ]:
ipcae <- data.frame(time=ipcae$date, expec=ipcae <- ipcae$median[ipcae$smoothed=='N'])

In [ ]:
ipcae$time <- as.Date(ipcae$time, format='%d/%m/%Y')

In [ ]:
ipcae$expec <- xts(ipcae$expec, order.by = ipcae$time)

In [ ]:
expec <- apply.monthly(ipcae$expec, FUN=colMeans)

In [ ]:
expec <- ts( expec, start = c( 2003, 12 ), freq=12);

Faça um gráfico da série:

# Taxa de juros real

Calcule a taxa de juros real ex-ante.

Faça um gráfico da série:

Calcule a primeira diferença da série:

Faça um gráfico da série:

## Testes de raiz unitária

Teste se a série é estacionária. Para isso, utilize os testes ADF, KPSS e PP.

#### ADF (Augmented-Dickey-Fuller)

$H_0$:

Qual é a conclusão do teste?

R:

### KPSS

$H_0$:

Qual é a conclusão do teste?

R:

### PP

$H_0$:

# VAR

Agora, vamos estimar o VAR. Para isso, utilizaremos o pacote 'vars':

In [ ]:
install.packages("vars")

In [ ]:
library(vars)

## Ordem do VAR

O primeiro passo é escolher a ordem do VAR. Os critérios de informação podem nos ajudar nessa etapa. Primeiro, crie um data frame com as variáveis:

In [ ]:
vardata <- ts.intersect( dreal, de, adm, liv )

Depois, encontre a melhor especificação de acordo com cada um dos critérios:

In [ ]:
selection <- VARselect( vardata, lag.max = 12, type = "const")

In [ ]:
selection$selection

No box do relatório do BCB ([aqui](https://www.bcb.gov.br/content/ri/relatorioinflacao/200406/RELINF200406-ri200406b8p.pdf)), a entidade monetária utiliza duas defasagens. Vamos seguir dessa forma para fins didáticos.

Vamos estimar o VAR(2):

In [ ]:
var = VAR( vardata, p = 2, type = "const")

In [ ]:
summary( var )

Podemos utilizar o pacote 'stargazer' para visualizar de uma forma alternativa o resultado da estimação:

In [ ]:
install.packages("stargazer")

In [ ]:
library(stargazer)

In [ ]:
stargazer::stargazer( var$varresult, type = "text", style = "aer", title = "VAR(2)", column.labels = c( "Juros", "Câmbio", "Preços Adm.", "Preços Livres" ) )

# VAR com Dummies sazonais

Estime agora um VAR com dummies sazonais:

In [ ]:
var2 = VAR( vardata, p = 2, type = "const", season = 12 )

Por que só temos 11 coeficientes? O que seria preciso fazer para termos 12 coeficientes?

# Desafios

Podemos melhorar o modelo com qual queremos projetar a inflação de serviços. Para isso, estime (i) um VAR com outras dummies (a serem descritas abaixo) e (ii) um VAR com outras dummies e outra ordem.

# VAR com outras dummies

Crie ao menos três dummies:

* d2008: valor 1 para observações entre 1/10/2008 e 31-03-2009 e zero caso contrário.
* d2014: valor 1 para observações entre 1/4/2014 e 31-12-2016 e zero caso contrário.
* dcovid: valor 1 para observações entre 1/01/2020 e 30-06-2020 e zero caso contrário.

Crie um objeto com as dummies:

In [ ]:
dummies = cbind( d2008, d2014, dcovid )

Estime o VAR:

# VAR com outras dummies e outra ordem

Nós seguimos o box do relatório do BCB quanto à ordem do VAR nos exercícios anteriores. Mas com os dados que temos hoje, será a melhor escolha? Estime dois outros modelos: um com a ordem com base no Hannan–Quinn e outro com base no Akaike.

## Ordem do VAR

In [ ]:
stargazer::stargazer( var5$varresult, type = "text", style = "aer", title = "VAR(2)", column.labels = c( "Juros", "Câmbio", "Preços Adm.", "Preços Livres" ) )


VAR(2)
                                                     y                      
                                 Juros    Câmbio   Preços Adm. Preços Livres
                                  (1)       (2)        (3)          (4)     
----------------------------------------------------------------------------
dreal.l1                       0.427***   -0.002      0.134        0.041    
                                (0.071)   (0.022)    (0.131)      (0.042)   
                                                                            
de.l1                            0.393   0.214***    -0.001        0.046    
                                (0.251)   (0.076)    (0.461)      (0.147)   
                                                                            
adm.l1                           0.027    0.023*    0.365***       0.022    
                                (0.039)   (0.012)    (0.072)      (0.023)   
                                                                    

# Vamos salvar os resultados!

Salve localmente o arquivo criado com a função 'save.image':

In [ ]:
save.image( file = "var.RData" )

**IMPORTANTE**: Faça o download do arquivo e salve-o para utilizá-lo no notebook da próxima aula.